```text
chen 
Sector 0, Block 0 (Manufacturer data):
DE74925E662804009010150100000000
Sector 0, Block 0 (Manufacturer data): DE74925E662804009010150100000000

ben ke
8A08B1E0D3080400D200000100000000

Sector 0, Block 0 (Manufacturer data): 0EA7F129712804009010150100000000
Sector 0, Block 0 (Manufacturer data): 0EA7F129712804009010150100000000

CED6E929D82804009010150100000000
# 202302150014
EEC4755E012804009010150100000000
# 202402150034

```

In [2]:
# 0xEE XOR 0xC4 XOR 0x75 XOR 0x5E
# use python3 to run this script

def xor(a, b):
    return a ^ b

a = 0xCE
b = 0xD6
c = 0xE9
d = 0x29
print(hex(xor(xor(xor(a, b), c), d)))


0xd8


Some possibilities to consider:

- [x] CRC32 hash (common for generating 8-character hex values)
- [ ] Custom XOR operation with a key
- [x] NO: Some form of truncated MD5/SHA hash
- [ ] A proprietary encryption algorithm
- [ ] try the numbers as a date

In [5]:
# here we have one question, 202402150034 is use some way to encoding to EEC4755E, the 202302150014 to CED6E929, here we want to know what maps to DE74925E?
# the 202402150034 pattern is the 2024 is the years, 0215 is the student major, 0034 is the student number. 
# can try use usually encoding way find the answer.

# like use the hash function to encoding 202402150034 to EEC4755E, 

import hashlib

def try_hash_methods(input_str):
    results = {}
    
    # Try MD5
    md5_hash = hashlib.md5(input_str.encode()).hexdigest().upper()
    results["MD5 first 8"] = md5_hash[:8]
    results["MD5 last 8"] = md5_hash[-8:]
    
    # Try SHA1
    sha1_hash = hashlib.sha1(input_str.encode()).hexdigest().upper()
    results["SHA1 first 8"] = sha1_hash[:8]
    results["SHA1 last 8"] = sha1_hash[-8:]
    
    # Try SHA256
    sha256_hash = hashlib.sha256(input_str.encode()).hexdigest().upper()
    results["SHA256 first 8"] = sha256_hash[:8]
    results["SHA256 last 8"] = sha256_hash[-8:]
    
    return results

# Verify with known examples
print("Testing 202402150034:")
results1 = try_hash_methods("202402150034")
for method, result in results1.items():
    if result == "EEC4755E":
        print(f"✓ Match found: {method} -> {result}")
    else:
        print(f"× {method} -> {result}")

print("\nTesting 202302150014:")
results2 = try_hash_methods("202302150014")
for method, result in results2.items():
    if result == "CED6E929":
        print(f"✓ Match found: {method} -> {result}")
    else:
        print(f"× {method} -> {result}")

# Now brute force search for DE74925E
print("\nSearching for ID that maps to DE74925E...")
target = "DE74925E"

# Try different years and student numbers with the same major
for year in range(2020, 2025):
    for num in range(1, 100):
        student_id = f"{year}0215{num:04d}"
        results = try_hash_methods(student_id)
        for method, result in results.items():
            if result == target:
                print(f"Found match: {student_id} using {method} -> {target}")
                
# If the above doesn't find it, try a wider range
if True:  # Change to True to execute this block
    for year in range(2000, 2030):
        for major in [215]:  # Keeping the same major
            for num in range(1, 50):  # Limit to 50 to avoid excessive runtime
                student_id = f"{year}{major:04d}{num:04d}"
                results = try_hash_methods(student_id)
                for method, result in results.items():
                    if result == target:
                        print(f"Found match: {student_id} using {method} -> {target}")

Testing 202402150034:
× MD5 first 8 -> 41229552
× MD5 last 8 -> C8D85568
× SHA1 first 8 -> 4B0023BF
× SHA1 last 8 -> F41E8CDC
× SHA256 first 8 -> E54E6846
× SHA256 last 8 -> D8DCE10D

Testing 202302150014:
× MD5 first 8 -> 1B775622
× MD5 last 8 -> 6903F2D8
× SHA1 first 8 -> 744A127F
× SHA1 last 8 -> 806C8E8C
× SHA256 first 8 -> DC95381D
× SHA256 last 8 -> 667397FD

Searching for ID that maps to DE74925E...


In [6]:
import zlib

def crc32_hash(input_str):
    # Calculate CRC32 hash and format as 8-character uppercase hex
    crc = zlib.crc32(input_str.encode())
    return format(crc & 0xFFFFFFFF, '08X')  # Ensure 8 chars with leading zeros

# Verify with known examples
print("Testing 202402150034:")
result1 = crc32_hash("202402150034")
print(f"CRC32 -> {result1}")
print(f"Match with EEC4755E: {result1 == 'EEC4755E'}")

print("\nTesting 202302150014:")
result2 = crc32_hash("202302150014")
print(f"CRC32 -> {result2}")
print(f"Match with CED6E929: {result2 == 'CED6E929'}")

# Now search for DE74925E
print("\nSearching for ID that maps to DE74925E...")
target = "DE74925E"

# Try with the same major (0215) but different years and numbers
for year in range(2000, 2030):
    for num in range(1, 100):
        student_id = f"{year}0215{num:04d}"
        result = crc32_hash(student_id)
        if result == target:
            print(f"Found match: {student_id} -> {target}")
            
# If no match found with the first approach, try different majors too
found = False
for year in range(2020, 2025):
    if found:
        break
    for major in range(100, 500):  # Try different majors
        if found:
            break
        for num in range(1, 100):
            student_id = f"{year}{major:04d}{num:04d}"
            result = crc32_hash(student_id)
            if result == target:
                print(f"Found match with different major: {student_id} -> {target}")
                found = True
                break

Testing 202402150034:
CRC32 -> A93F6E7F
Match with EEC4755E: False

Testing 202302150014:
CRC32 -> FF686134
Match with CED6E929: False

Searching for ID that maps to DE74925E...


In [7]:
import zlib
import struct

def crc32_hash_str(input_str):
    """Standard CRC32 on a string"""
    crc = zlib.crc32(input_str.encode())
    return format(crc & 0xFFFFFFFF, '08X')

def crc32_hash_int(input_int):
    """CRC32 treating the input as a binary integer"""
    # Convert integer to bytes
    # Try both little endian and big endian
    bytes_le = struct.pack("<Q", input_int)  # Little endian
    bytes_be = struct.pack(">Q", input_int)  # Big endian
    
    crc_le = zlib.crc32(bytes_le)
    crc_be = zlib.crc32(bytes_be)
    
    return {
        "LE": format(crc_le & 0xFFFFFFFF, '08X'),
        "BE": format(crc_be & 0xFFFFFFFF, '08X')
    }

def crc32_hash_bytes(input_int):
    """CRC32 after converting the integer to its minimal byte representation"""
    # Convert to minimal bytes (no padding)
    bytes_data = input_int.to_bytes((input_int.bit_length() + 7) // 8, 'big')
    crc = zlib.crc32(bytes_data)
    return format(crc & 0xFFFFFFFF, '08X')

# Test known examples
test_ids = ["202402150034", "202302150014"]
for id_str in test_ids:
    id_int = int(id_str)
    
    print(f"Testing {id_str}:")
    print(f"String CRC32: {crc32_hash_str(id_str)}")
    
    int_results = crc32_hash_int(id_int)
    print(f"Integer LE CRC32: {int_results['LE']}")
    print(f"Integer BE CRC32: {int_results['BE']}")
    
    print(f"Minimal bytes CRC32: {crc32_hash_bytes(id_int)}")
    print()

# Now search for DE74925E
target = "DE74925E"
print(f"Searching for ID that maps to {target}...")

# Search using the method(s) that matched the test cases
# Try both standard major (0215) and other values
for year in range(2000, 2030):
    for major in [215]:  # Can expand if needed
        for num in range(1, 100):
            id_str = f"{year}{major:04d}{num:04d}"
            id_int = int(id_str)
            
            # Try each hash method
            str_hash = crc32_hash_str(id_str)
            if str_hash == target:
                print(f"Found match (string): {id_str} -> {target}")
                
            int_results = crc32_hash_int(id_int)
            if int_results['LE'] == target:
                print(f"Found match (int LE): {id_str} -> {target}")
            if int_results['BE'] == target:
                print(f"Found match (int BE): {id_str} -> {target}")
                
            bytes_hash = crc32_hash_bytes(id_int)
            if bytes_hash == target:
                print(f"Found match (minimal bytes): {id_str} -> {target}")

Testing 202402150034:
String CRC32: A93F6E7F
Integer LE CRC32: 1BA3744B
Integer BE CRC32: 3A6C7F45
Minimal bytes CRC32: 996C5731

Testing 202302150014:
String CRC32: FF686134
Integer LE CRC32: A063D4E7
Integer BE CRC32: 6C63392D
Minimal bytes CRC32: CF631159

Searching for ID that maps to DE74925E...


In [8]:
import zlib
import struct
import hashlib

def crc32_hash_int(input_int):
    """CRC32 treating the input as a binary integer"""
    # Convert to minimal bytes (no padding)
    bytes_data = input_int.to_bytes((input_int.bit_length() + 7) // 8, 'big')
    crc = zlib.crc32(bytes_data)
    return format(crc & 0xFFFFFFFF, '08X')

def md5_hash_int(input_int):
    """MD5 hash of the integer's byte representation"""
    bytes_data = input_int.to_bytes((input_int.bit_length() + 7) // 8, 'big')
    md5_hash = hashlib.md5(bytes_data).hexdigest().upper()
    return md5_hash

def sha1_hash_int(input_int):
    """SHA1 hash of the integer's byte representation"""
    bytes_data = input_int.to_bytes((input_int.bit_length() + 7) // 8, 'big')
    sha1_hash = hashlib.sha1(bytes_data).hexdigest().upper()
    return sha1_hash

# Test known examples with integer-based hashing
test_ids = [202402150034, 202302150014]
for id_int in test_ids:
    print(f"Testing {id_int}:")
    print(f"CRC32: {crc32_hash_int(id_int)}")
    print(f"MD5 (first 8 chars): {md5_hash_int(id_int)[:8]}")
    print(f"SHA1 (first 8 chars): {sha1_hash_int(id_int)[:8]}")
    print()

# Target hash
target = "DE74925E"
print(f"Searching for ID that maps to {target}...")

# Pattern: YYYYMMDDXXXX where XXXX is a sequential number
# More structured search focusing on realistic date patterns
found = False
hash_types = ["CRC32", "MD5", "SHA1"]

for year in range(2020, 2025):  # More focused year range
    for month in range(1, 13):
        for day in range(1, 32):  # We could add date validation for efficiency
            # Create base ID in format YYYYMMDD
            base_id = year * 10000 + month * 100 + day
            
            # Search a range of sequential numbers
            for seq in range(1, 10000):
                # Two common patterns: YYYYMMDDXXXX or YYYYMMDDSXXX (where S is a separator digit)
                for pattern in [1, 2]:
                    if pattern == 1:
                        # Pattern: YYYYMMDDXXXX
                        id_int = base_id * 10000 + seq
                    else:
                        # Pattern: YYYYMMDDSXXX (where S is often 0)
                        separator = 0  # Could try other separators
                        id_int = base_id * 10000 + separator * 1000 + seq % 1000
                    
                    # Try CRC32
                    crc32_result = crc32_hash_int(id_int)
                    if crc32_result == target:
                        print(f"Found match (CRC32): {id_int} -> {target}")
                        found = True
                    
                    # Try truncated MD5
                    md5_result = md5_hash_int(id_int)[:8]
                    if md5_result == target:
                        print(f"Found match (MD5 truncated): {id_int} -> {target}")
                        found = True
                    
                    # Try truncated SHA1
                    sha1_result = sha1_hash_int(id_int)[:8]
                    if sha1_result == target:
                        print(f"Found match (SHA1 truncated): {id_int} -> {target}")
                        found = True
                    
                    if found:
                        break
                
                # Print progress every 100,000 IDs checked
                if seq % 100000 == 0:
                    print(f"Checked up to {id_int}...")
                
                if found:
                    break
            
            if found:
                break
        
        if found:
            break
    
    if found:
        break

if not found:
    print("No match found in the specified range. Try expanding the search or using different patterns.")

Testing 202402150034:
CRC32: 996C5731
MD5 (first 8 chars): BCB543C6
SHA1 (first 8 chars): B0567CB8

Testing 202302150014:
CRC32: CF631159
MD5 (first 8 chars): BBE9084A
SHA1 (first 8 chars): F1B670E4

Searching for ID that maps to DE74925E...
No match found in the specified range. Try expanding the search or using different patterns.
